# FINAL VERSION

User: santoshdkolur
Password : santoshdkolur

Locally hosted database using XAMPP.

In [2]:
import cv2
import pickle
import numpy as np
import face_recognition,datetime
import pymysql
import pandas as pd
import datetime
from os import listdir

In [3]:
connection = pymysql.connect(host="localhost",user="santoshdkolur",passwd="santoshdkolur",database="attendance" )
cursor = connection.cursor()

In [11]:
def trainData():
    known_name=[]
    known_face_encodings = []
    semester=input("Enter the Semester:Eg. 6 for 6th semester")
    section=input("Enter the Section:")
    sql='create table '+str(section)+str(semester)+' (serial int(11),srn varchar(8),status int(11),time timestamp DEFAULT CURRENT_TIMESTAMP,date date)'
    cursor.execute(sql)
    path=input("Enter the part to the training data:")
    path='/Mini_Train/'            #This parameter can be taken as input.
    for ele in listdir(path):
        img=cv2.imread(path+ele)
        img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)    #Resize to half the original size
        img = img[:, :, ::-1]                            #Convert from BGR(OpenCV) to RGB(Face_Recognition)
        face_locations = face_recognition.face_locations(img)
        face_encodings = face_recognition.face_encodings(img,face_locations)
        if(len(face_encodings)!=0):
            known_name.append(ele)
            known_face_encodings.append(face_encodings[0])
    known_face_names= known_name
    
    #Store face encodings data
    fdata='trained_data/'+section+semester+'_data.pkl'        #File name for the stored data
    fname='trained_data/'+section+semester+'_name.pkl'
    with open(fdata,"wb") as f:          #Store MultiDimensional Array
        pickle.dump(known_face_encodings,f)
    with open(fname,"wb") as f:
        pickle.dump(known_face_names,f)

In [7]:
def detectFace():
    semester=input("Enter the Semester:Eg. 6 for 6th semester")
    section=input("Enter the Section:")
    fdata='trained_data/'+section+semester+'_data.pkl'        #File name for the stored data
    fname='trained_data/'+section+semester+'_name.pkl'
    
    
    #Open saved facevectors for 6th sem F sec.
    #Once we get the faces of around 45 people from 3 different class, we can retrieve the data while running to detection script
    with open(fdata,"rb") as f:
        known_face_encodings=pickle.load(f)
    with open(fname,"rb") as f:
        known_face_names=pickle.load(f)
        
    detected_face={}
    
    # Initialize some variables
    serial=cursor.execute('select serial from f6 order by serial desc;')
    video_capture=cv2.VideoCapture(0)
    known_face_encodings=known_face_encodings
    face_locations = []

    face_names = []
    matches=[]
    i=1;
    process_this_frame = True

    while True:
        # Grab a single frame of video
        ret, frame = video_capture.read()
        face_encodings = []

        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
            # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
               # matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                #name = "Unknown"+str(i)

            # # If a match was found in known_face_encodings, just use the first one.
               # if True in matches:
               #     first_match_index = matches.index(True)
                #    name = known_face_names[first_match_index]
                 #   print(name)
                  #  face_names.append(name)
                   # print(matches)

            # Or instead, use the known face with the smallest distance to the new face
                #else:
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
               # if matches[best_match_index]:
                name = known_face_names[best_match_index]  #Name is basically their srn

                if(np.min(face_distances)>0.5):
                    name="Not Recognized"
                else:
                    print('Inside...')
                    print(list(detected_face.keys()))
                    if(name[:8] not in list(detected_face.keys())):
                        q="select status from 'F6' where srn="+str(name[:8])+"and date="+str(datetime.datetime.now())[0:10]+" order by time DESC LIMIT 1;"
                        if(len(pd.read_sql(q,connection))==0):
                            status=1
                        else:
                            status=!cursor.execute(q)
                        sql = "INSERT INTO `F6` (srn,status,date,serial) values('"+name[:8]+"','"+str(status)+"','"+str(datetime.datetime.now())[0:10]+"','"+str(serial)+"')" #Other parameters such as time and date are automatically updated
                        serial=serial+1
                        cursor.execute(sql)
                        print(sql)
                        name=name[:8]
                        print(name)
                        detected_face[name]=datetime.datetime.now()
                        connection.commit()                 #Add data to the database!
                face_names.append(name)


        process_this_frame = not process_this_frame

        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.8, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        for ele in list(detected_face.keys()):
            a=datetime.datetime.now()
            b=detected_face[ele]
            if(((a-b).seconds/60)>5):
                detected_face.pop(ele)



    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()

In [9]:
def updateAttendance():   
#Implementing for one class as of now. We'll add more classes later!
    
    retrieve='select * from f6 where date=\''+str(datetime.datetime.now())[0:10]+'\''
    df=pd.read_sql(retrieve,connection)
    df=df.sort_values(['srn','time'],ascending=[True,True])
    srn=set(df['srn'])

    for ele in srn:
        student=df.loc[df['srn']==ele]
        #We now have the data of particular student for that day. 
        #The times at which his face was logged.
        #Figure out a way to reward attendace from this data. 
    
    #Login_details=[df[df['srn']==ele] for ele in df['srn']]
    

In [ ]:
def main():
    print("Select one of the below options:")
    choice=int(input("1. Train data\n2. Start logging\n3. Update Attendance"))
    if(choice==1):
        trainData()
    else if(choice==2):
        detectFace()
    else if(choice==3):
        updateAttendance();
    else:
        print("Invalid Input")
    